In [ ]:
import warnings
import datetime

from importlib import reload
from pathlib import Path
from typing import Union, Tuple, List, Dict
from tqdm import tqdm

import numpy as np
from scipy.optimize import Bounds
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


from sdm_eurec4a import RepositoryPath, get_git_revision_hash
from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    adjust_lightness_array,
    ncols_nrows_from_N,
    label_from_attrs,
)
import sdm_eurec4a.input_processing.models as smodels
from sdm_eurec4a.reductions import mean_and_stderror_of_mean
from sdm_eurec4a.identifications import match_clouds_and_cloudcomposite
from sdm_eurec4a.conversions import msd_from_psd_dataarray, psd_from_msd_dataarray

warnings.filterwarnings("ignore")

default_colors = set_custom_rcParams()
dark_colors = adjust_lightness_array(default_colors, 0.7)

from sdm_eurec4a import slurm_cluster

data_dir = RepositoryPath("levante").data_dir

fig_dir = Path("107_v4.2")
fig_dir.mkdir(exist_ok=True, parents=False)

output_dir = data_dir / Path("model/input_v4.2")
output_dir.mkdir(exist_ok=True)

In [ ]:
# client, cluster = slurm_cluster.init_dask_slurm_cluster(
#     scale=1, processes=16, walltime="00:35:00", memory="16GB",
#     scheduler_options={"dashboard_address": ":8686"},
# )

## Import observation data

In [ ]:
cloud_composite = xr.open_dataset(
    data_dir / Path("observation/cloud_composite/processed/cloud_composite_SI_units_20241025.nc"),
)
identified_clusters = xr.open_dataset(
    data_dir
    / Path(
        "observation/cloud_composite/processed/identified_clusters/identified_clusters_rain_mask_5.nc"
    )
)
identified_clusters = identified_clusters.swap_dims({"time": "cloud_id"})

# attrs = cloud_composite["radius"].attrs.copy()
# attrs.update({"units": "µm"})
# cloud_composite["radius"] = cloud_composite["radius"]
# cloud_composite["radius_micro"] = 1e6 * cloud_composite["radius"]
# cloud_composite["radius"].attrs = attrs

cloud_composite["radius2D"] = cloud_composite["radius"].expand_dims(time=cloud_composite["time"])
cloud_composite = cloud_composite.transpose("radius", ...)


# cloud_composite = cloud_composite.sel(radius=slice(10e-6, None))

identified_clusters = identified_clusters.where(
    (
        (identified_clusters.duration.dt.seconds >= 3)
        & (identified_clusters.altitude < 1200)
        & (identified_clusters.altitude > 500)
    ),
    drop=True,
)
# identified_clusters = identified_clusters.isel(cloud_id=slice(0, 20))

If we want to coarsen the results, we need to make sure to apply the coarsening on the **NON** normalized data.
Then we can normalized afterwards again

In [ ]:
lower_radius_split = 95e-6  # 50 µm
higher_radius_split = 5e-3  # 1 mm
coarsen_factor = 3

# lower_radius_split = 90e-6  # 50 µm
# higher_radius_split = 0.5e-3  # 1 mm
# coarsen_factor = 3

coarse_composite = cloud_composite.sel(radius=slice(lower_radius_split, higher_radius_split)).copy()

# make sure to have non normalized data to be coarsened
# otherwise, the sum will not be conserved
coarse_composite["particle_size_distribution"] = (
    coarse_composite["particle_size_distribution"] * coarse_composite["bin_width"]
)
coarse_composite["mass_size_distribution"] = (
    coarse_composite["mass_size_distribution"] * coarse_composite["bin_width"]
)

# use mean for radius and radius2D
coarse_composite_radius = coarse_composite["radius"].coarsen(radius=coarsen_factor).mean()
coarse_composite_radius2D = coarse_composite["radius2D"].coarsen(radius=coarsen_factor).mean()
# use the sum for the rest
coarse_composite = coarse_composite.coarsen(radius=coarsen_factor).sum()

coarse_composite["radius"] = coarse_composite_radius
coarse_composite["radius2D"] = coarse_composite_radius2D
coarse_composite["diameter"] = 2 * coarse_composite["radius"]

# make sure to have normalized data again
coarse_composite["particle_size_distribution"] = (
    coarse_composite["particle_size_distribution"] / coarse_composite["bin_width"]
)
coarse_composite["mass_size_distribution"] = (
    coarse_composite["mass_size_distribution"] / coarse_composite["bin_width"]
)

coarse_composite["particle_size_distribution"].attrs = dict(
    long_name="Number concentration",
    unit=cloud_composite["particle_size_distribution"].attrs["unit"],
)
coarse_composite["mass_size_distribution"].attrs = dict(
    long_name="Mass concentration",
    unit=cloud_composite["mass_size_distribution"].attrs["unit"],
)

# merge the two composites with higher resoltion at small radii
# and lower resolution at large radii
coarse_composite = xr.merge(
    [
        coarse_composite.sel(radius=slice(lower_radius_split, higher_radius_split)),
        cloud_composite.sel(radius=slice(None, lower_radius_split)),
        cloud_composite.sel(radius=slice(higher_radius_split, None)),
    ]
)


# Test liquid water content is conserved
np.testing.assert_allclose(
    (coarse_composite["bin_width"] * coarse_composite["mass_size_distribution"]).sum("radius"),
    (cloud_composite["bin_width"] * cloud_composite["mass_size_distribution"]).sum("radius"),
    rtol=0.001,
)
# Test particle concentration is conserved
np.testing.assert_allclose(
    (coarse_composite["bin_width"] * coarse_composite["particle_size_distribution"]).sum("radius"),
    (cloud_composite["bin_width"] * cloud_composite["particle_size_distribution"]).sum("radius"),
    rtol=0.001,
)

## Boundaries for the fitting parameters

Please note, that the value of the fitting parameter are very very sensible!!!

They need to be adjusted with a lot of effort to make the fit work well!

Better bounds are:
````
class PSDBounds:

    def __init__(self):
        self.mu1 = np.array([1e-6, 14e-6, 50e6])
        self.mu2 = np.array([0.1e-3, 0.3e-3, 0.5e-3])
        self.sigma1 = np.array([1.5, 1.6, 3.0])
        self.sigma2 = np.array([1.4, 1.5, 2.0])
        self.scale1 = np.array([1e-20, 1e13, 1e16])
        self.scale2 = np.array([1e-20, 1e3, 1e10])
class MSDBounds:

    def __init__(self):
        self.mu1 = np.array([1e-6, 14e-6, 50e6])
        self.mu2 = np.array([0.1e-3, 0.3e-3, 0.5e-3])
        self.sigma1 = np.array([1.3, 1.5, 3.5])
        self.sigma2 = np.array([1.2, 1.4, 2.0])
        self.scale1 = np.array([1e-20, 1e-1, 1e2])
        self.scale2 = np.array([1e-20, 1e-4, 1e1])

````


In [ ]:
class PSDBounds:

    def __init__(self):
        self.mu1 = np.array([1e-6, 14e-6, 50e6])  # µm
        self.mu2 = np.array([0.1e-3, 0.3e-3, 0.5e-3])  # µm
        self.sigma1 = np.array([1.5, 1.6, 3.0])  # ??
        self.sigma2 = np.array([1.4, 1.5, 2.0])  # ??
        self.scale1 = np.array([1e-20, 1e13, 1e16])  # (Number of droplets)
        self.scale2 = np.array([1e-20, 1e3, 1e10])  # (Number of droplets)
        ## to omit the scale factor issue, we could try and set them higher for a better fit
        ## (see issue #131)
        # self.scale1 = np.array([1e-20, 2e13, 2e16])
        # self.scale2 = np.array([1e-20, 2e3, 2e10])

    def get_x0_bounds(
        self,
    ) -> Tuple[np.ndarray, Bounds]:

        x0 = np.array(
            [
                self.mu1[1],
                self.sigma1[1],
                self.scale1[1],
                self.mu2[1],
                self.sigma2[1],
                self.scale2[1],
            ]
        )
        lower_bound = np.array(
            [
                self.mu1[0],
                self.sigma1[0],
                self.scale1[0],
                self.mu2[0],
                self.sigma2[0],
                self.scale2[0],
            ]
        )
        upper_bound = np.array(
            [
                self.mu1[2],
                self.sigma1[2],
                self.scale1[2],
                self.mu2[2],
                self.sigma2[2],
                self.scale2[2],
            ]
        )

        bounds = Bounds(
            lb=lower_bound,
            ub=upper_bound,
        )

        return x0, bounds

    @property
    def bounds(self):
        x0, bounds = self.get_x0_bounds()
        return bounds

    @property
    def x0(self):
        x0, bounds = self.get_x0_bounds()
        return x0


class MSDBounds:

    def __init__(self):
        self.mu1 = np.array([1e-6, 14e-6, 50e6])  # µm
        self.mu2 = np.array([0.1e-3, 0.3e-3, 0.5e-3])  # µm
        self.sigma1 = np.array([1.3, 1.5, 3.5])  # ??
        self.sigma2 = np.array([1.2, 1.4, 2.0])  # ??
        self.scale1 = np.array([1e-20, 1e-1, 1e2])  # kg (mass of all droplets)
        self.scale2 = np.array([1e-20, 1e-4, 1e1])  # kg (mass of all droplets)
        ## to omit the scale factor issue, we could try and set them higher for a better fit
        ## (see issue #131)
        # self.scale1 = np.array([1e-20, 2e-1, 2e2])
        # self.scale2 = np.array([1e-20, 2e-4, 2e1])

    def get_x0_bounds(
        self,
    ) -> Tuple[np.ndarray, Bounds]:

        x0 = np.array(
            [
                self.mu1[1],
                self.sigma1[1],
                self.scale1[1],
                self.mu2[1],
                self.sigma2[1],
                self.scale2[1],
            ]
        )
        lower_bound = np.array(
            [
                self.mu1[0],
                self.sigma1[0],
                self.scale1[0],
                self.mu2[0],
                self.sigma2[0],
                self.scale2[0],
            ]
        )
        upper_bound = np.array(
            [
                self.mu1[2],
                self.sigma1[2],
                self.scale1[2],
                self.mu2[2],
                self.sigma2[2],
                self.scale2[2],
            ]
        )

        bounds = Bounds(
            lb=lower_bound,
            ub=upper_bound,
        )

        return x0, bounds

    @property
    def bounds(self):
        x0, bounds = self.get_x0_bounds()
        return bounds

    @property
    def x0(self):
        x0, bounds = self.get_x0_bounds()
        return x0

## Chose the correct representation of the log normal disribution.

Because there are multiple functions which are used as log normal distribution, the correct usage and transformation is crucial!!


In [ ]:
def dict_to_DataArray(d: Dict, new_coords: Dict) -> xr.Dataset:
    new_dims = list(new_coords.keys())

    d_new = {}
    for key in d:
        value = d[key]

        # if value is a DataArray, list or numpy array, we can direclty create a DataArray
        if isinstance(value, (np.ndarray, list, xr.DataArray)):
            d_new[key] = xr.DataArray(
                value,
                coords=new_coords,
            )
        # if value is a number, we create a list with this number
        elif isinstance(value, float):
            # create a list, if value is only a number
            d_new[key] = xr.DataArray(
                [value],
                coords=new_coords,
                dims=new_dims,
            )

    result = xr.Dataset(
        # coords= new_coords,
        data_vars=d_new,
    )
    return result

### Prepare individual datasets to be used for the fit 

In [ ]:
list_lwc = []
list_lwc_sem = []
list_lwc_50um = []
list_lwc_50um_sem = []

list_nbc = []
list_nbc_sem = []
list_nbc_50um = []
list_nbc_50um_sem = []

for cloud_id in tqdm(identified_clusters["cloud_id"]):
    cc = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=coarse_composite,
    )

    lwc = cc["liquid_water_content"]

    lwc_mean, lwc_sem = mean_and_stderror_of_mean(lwc, dims=("time",))
    list_lwc.append(lwc_mean)
    list_lwc_sem.append(lwc_sem)

    lwc_above_50um = (
        (cc["mass_size_distribution"] * cc["bin_width"]).sel(radius=slice(50e-6, None)).sum("radius")
    )
    lwc_above_50um_mean, lwc_above_50um_sem = mean_and_stderror_of_mean(lwc_above_50um, dims=("time",))
    list_lwc_50um.append(lwc_above_50um_mean)
    list_lwc_50um_sem.append(lwc_above_50um_sem)

    nbc = (cc["particle_size_distribution"] * cc["bin_width"]).sum("radius")

    nbc_mean, nbc_sem = mean_and_stderror_of_mean(nbc, dims=("time",))
    list_nbc.append(nbc_mean)
    list_nbc_sem.append(nbc_sem)

    nbc_above_50um = (
        (cc["particle_size_distribution"] * cc["bin_width"]).sel(radius=slice(50e-6, None)).sum("radius")
    )
    nbc_above_50um_mean, nbc_above_50um_sem = mean_and_stderror_of_mean(nbc_above_50um, dims=("time",))
    list_nbc_50um.append(nbc_above_50um_mean)
    list_nbc_50um_sem.append(nbc_above_50um_sem)


da_lwc = xr.concat(
    list_lwc,
    dim="cloud_id",
)
da_lwc.attrs = dict(
    long_name="Liquid water content",
    units="g m^{-3}",
)

da_lwc_sem = xr.concat(
    list_lwc_sem,
    dim="cloud_id",
)
da_lwc_sem.attrs = dict(
    long_name="Standard error of the mean of the liquid water content",
    units="g m^{-3}",
)

da_lwc_50um = xr.concat(
    list_lwc_50um,
    dim="cloud_id",
)
da_lwc_50um.attrs = dict(
    long_name="Liquid water content above 50 µm",
    units="g m^{-3}",
)

da_lwc_50um_sem = xr.concat(
    list_lwc_50um_sem,
    dim="cloud_id",
)
da_lwc_50um_sem.attrs = dict(
    long_name="Standard error of the mean of the liquid water content above 50 µm",
    units="g m^{-3}",
)

da_nbc = xr.concat(
    list_nbc,
    dim="cloud_id",
)
da_nbc.attrs = dict(
    long_name="Number concentration",
    units="m^{-3}",
)

da_nbc_sem = xr.concat(
    list_nbc_sem,
    dim="cloud_id",
)
da_nbc_sem.attrs = dict(
    long_name="Standard error of the mean of the number concentration",
    units="m^{-3}",
)

da_nbc_50um = xr.concat(
    list_nbc_50um,
    dim="cloud_id",
)
da_nbc_50um.attrs = dict(
    long_name="Number concentration above 50 µm",
    units="m^{-3}",
)

da_nbc_50um_sem = xr.concat(
    list_nbc_50um_sem,
    dim="cloud_id",
)
da_nbc_50um_sem.attrs = dict(
    long_name="Standard error of the mean of the number concentration above 50 µm",
    units="m^{-3}",
)


ds_observations = xr.Dataset(
    dict(
        liquid_water_content=da_lwc,
        liquid_water_content_sem=da_lwc_sem,
        liquid_water_content_50um=da_lwc_50um,
        liquid_water_content_50um_sem=da_lwc_50um_sem,
        particle_size_distribution=da_nbc,
        particle_size_distribution_sem=da_nbc_sem,
        particle_size_distribution_50um=da_nbc_50um,
        particle_size_distribution_50um_sem=da_nbc_50um_sem,
    )
)

100%|██████████| 154/154 [00:04<00:00, 35.65it/s]


### Define t,w for the fitting algorithm

In [ ]:
r = np.geomspace(0.1e-6, 3e-3, 100)

t_test = xr.DataArray(data=r, coords={"radius": r}, dims=["radius"])

# compute the weighting for each bin
# IMPORTANT: Divide by 2 to have the correct bin_width. Missing this, will result in a bin width of double the size!
w_test = (t_test - t_test.shift(radius=2)).shift(radius=-1) / 2
w_test = w_test.interpolate_na("radius", method="linear", fill_value="extrapolate")

### Fit with weights

##### Fit in number concentration space

In [ ]:
list_parameters = []
fitted_data = []


for cloud_id in tqdm(identified_clusters["cloud_id"].values):

    # extract the cloud composite data for the cloud
    cc = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=coarse_composite,
    )

    # use the particle size distribution and the radius
    psd = cc["particle_size_distribution"]  # .sel(radius = slice(15e-6, None))
    radius = cc["radius"].expand_dims(time=cc["time"])  # .sel(radius = slice(15e-6, None))

    # make sure to have the same order of dimensions
    psd = psd.transpose("time", "radius")
    radius = radius.transpose("time", "radius")

    # fit the double log-normal distribution
    double_ln = smodels.CleoDoubleLnNormalFit(
        name="PSD",
        x0=PSDBounds().x0,
        bounds=PSDBounds().bounds,
        t_train=radius.mean("time", skipna=True),
        y_train=psd.mean("time", skipna=True),
        fit_kwargs=dict(
            loss="linear",
            kwargs=dict(
                # parameter_space= 'geometric',
                # x_space = 'linear',
                # density_scaled = False,
            ),
        ),
    )
    double_ln.fit(3)

    # save the parameters
    list_parameters.append(
        dict_to_DataArray(
            d=double_ln.parameters,
            new_coords=dict(
                cloud_id=[
                    cloud_id,
                ]
            ),
        )
    )
    # predict the number concentration
    dimension, prediction = double_ln.predict(t_test)

    prediction = prediction.expand_dims(cloud_id=(cloud_id,))

    # # we can also use only the radii where we have data:
    # radii_measured = cc["particle_size_distribution"].mean("time") > 0
    # radius_end = cc["radius"].sel(radius=radii_measured).max().values
    # radius_start = cc["radius"].sel(radius=radii_measured).min().values
    # radius_end = 1.5 * radius_end
    # prediction = prediction.where(dimension >= radius_start, other = np.nan)
    # prediction = prediction.where(dimension <= radius_end, other = np.nan)

    fitted_data.append(prediction)


# create a data array with the fitted number concentration
particle_size_distribution_fitted_data = xr.concat(
    fitted_data,
    dim="cloud_id",
)
particle_size_distribution_parameters = xr.concat(
    list_parameters,
    dim="cloud_id",
)

 26%|██▌       | 40/154 [00:01<00:05, 20.20it/s]

100%|██████████| 154/154 [00:07<00:00, 20.78it/s]


In [ ]:
details = ""
details += "Parameters of the double log-normal distribution fitted to the particle size distribution.\n"
details += "The fitting function was set in the CleoDoubleLnNormalFit class.\n"
details += "No weighting used in the cost function.\n"
details += "A coarsened input data was used.\n"
details += "The loss function was set to 'linear'.\n"

particle_size_distribution_parameters.attrs = {
    "description": "parameters of a double log-normal distribution fitted to the particle size distribution",
    "details": details,
    "creation_time": datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%d %H:%M:%S"),
    "author": "Nils Niebaum",
    "email": "nils-ole.niebaum@mpimet.mpg.de",
    "institution": "Max Planck Institute for Meteorology",
    "github_repository": "https://github.com/nilsnevertree/sdm-eurec4a",
    "git_commit": get_git_revision_hash(),
}

##### Square $^2$

In [ ]:
list_parameters = []
fitted_data = []

for cloud_id in tqdm(identified_clusters["cloud_id"].data):

    # extract the cloud composite data for the cloud
    cc = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=coarse_composite,
    )

    # use the particle size distribution and the radius
    psd = cc["particle_size_distribution"]  # .sel(radius = slice(15e-6, None))
    radius = cc["radius"].expand_dims(time=cc["time"])  # .sel(radius = slice(15e-6, None))

    # make sure to have the same order of dimensions
    psd = psd.transpose("time", "radius")
    radius = radius.transpose("time", "radius")

    # fit the double log-normal distribution
    double_ln = smodels.CleoDoubleLnNormalFit(
        name="PSD weighted",
        x0=PSDBounds().x0,
        bounds=PSDBounds().bounds,
        t_train=radius.mean("time", skipna=True),
        y_train=psd.mean("time", skipna=True),
        fit_kwargs=dict(
            loss="linear",
            kwargs=dict(
                # parameter_space= 'geometric',
                # x_space = 'linear',
                # density_scaled = False,
            ),
        ),
        t_weight_power=2,
    )
    double_ln.fit(3)

    # save the parameters
    list_parameters.append(
        dict_to_DataArray(
            d=double_ln.parameters,
            new_coords=dict(
                cloud_id=[
                    cloud_id,
                ]
            ),
        )
    )
    # predict the number concentration
    dimension, prediction = double_ln.predict(t_test)

    prediction = prediction.expand_dims(cloud_id=(cloud_id,))

    # # we can also use only the radii where we have data:
    # radii_measured = cc["particle_size_distribution"].mean("time") > 0
    # radius_end = cc["radius"].sel(radius=radii_measured).max().values
    # radius_start = cc["radius"].sel(radius=radii_measured).min().values
    # radius_end = 1.5 * radius_end
    # prediction = prediction.where(dimension >= radius_start, other = np.nan)
    # prediction = prediction.where(dimension <= radius_end, other = np.nan)

    fitted_data.append(prediction)


# create a data array with the fitted number concentration
weighted2_particle_size_distribution_fitted_data = xr.concat(
    fitted_data,
    dim="cloud_id",
)
weighted2_particle_size_distribution_parameters = xr.concat(
    list_parameters,
    dim="cloud_id",
)

100%|██████████| 154/154 [00:07<00:00, 19.79it/s]


In [ ]:
details = ""
details += "Parameters of the double log-normal distribution fitted to the particle size distribution.\n"
details += "The fitting function was set in the CleoDoubleLnNormalFit class.\n"
details += "Weighting by the radius to power of 2 (surface) used in the cost function.\n"
details += "A coarsened input data was used.\n"
details += "The loss function was set to 'linear'.\n"

weighted2_particle_size_distribution_parameters.attrs = {
    "description": "parameters of a double log-normal distribution fitted to the particle size distribution",
    "details": details,
    "creation_time": datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%d %H:%M:%S"),
    "author": "Nils Niebaum",
    "email": "nils-ole.niebaum@mpimet.mpg.de",
    "institution": "Max Planck Institute for Meteorology",
    "github_repository": "https://github.com/nilsnevertree/sdm-eurec4a",
    "git_commit": get_git_revision_hash(),
}

##### Cubic $^3$

In [ ]:
list_parameters = []
fitted_data = []

for cloud_id in tqdm(identified_clusters["cloud_id"].data):

    # extract the cloud composite data for the cloud
    cc = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=coarse_composite,
    )

    # use the particle size distribution and the radius
    psd = cc["particle_size_distribution"]  # .sel(radius = slice(15e-6, None))
    radius = cc["radius"].expand_dims(time=cc["time"])  # .sel(radius = slice(15e-6, None))

    # make sure to have the same order of dimensions
    psd = psd.transpose("time", "radius")
    radius = radius.transpose("time", "radius")

    # fit the double log-normal distribution
    double_ln = smodels.CleoDoubleLnNormalFit(
        name="PSD weighted",
        x0=PSDBounds().x0,
        bounds=PSDBounds().bounds,
        t_train=radius.mean("time", skipna=True),
        y_train=psd.mean("time", skipna=True),
        fit_kwargs=dict(
            loss="linear",
            kwargs=dict(
                # parameter_space= 'geometric',
                # x_space = 'linear',
                # density_scaled = False,
            ),
        ),
        t_weight_power=3,
    )
    double_ln.fit(3)

    # save the parameters
    list_parameters.append(
        dict_to_DataArray(
            d=double_ln.parameters,
            new_coords=dict(
                cloud_id=[
                    cloud_id,
                ]
            ),
        )
    )
    # predict the number concentration
    dimension, prediction = double_ln.predict(t_test)

    prediction = prediction.expand_dims(cloud_id=(cloud_id,))

    # # we can also use only the radii where we have data:
    # radii_measured = cc["particle_size_distribution"].mean("time") > 0
    # radius_end = cc["radius"].sel(radius=radii_measured).max().values
    # radius_start = cc["radius"].sel(radius=radii_measured).min().values
    # radius_end = 1.5 * radius_end
    # prediction = prediction.where(dimension >= radius_start, other = np.nan)
    # prediction = prediction.where(dimension <= radius_end, other = np.nan)

    fitted_data.append(prediction)


# create a data array with the fitted number concentration
weighted3_particle_size_distribution_fitted_data = xr.concat(
    fitted_data,
    dim="cloud_id",
)
weighted3_particle_size_distribution_parameters = xr.concat(
    list_parameters,
    dim="cloud_id",
)

100%|██████████| 154/154 [00:07<00:00, 21.99it/s]


In [ ]:
details = ""
details += "Parameters of the double log-normal distribution fitted to the particle size distribution.\n"
details += "The fitting function was set in the CleoDoubleLnNormalFit class.\n"
details += "Weighting by the radius to power of 3 (volume) used in the cost function.\n"
details += "A coarsened input data was used.\n"
details += "The loss function was set to 'linear'.\n"

weighted3_particle_size_distribution_parameters.attrs = {
    "description": "parameters of a double log-normal distribution fitted to the particle size distribution",
    "details": details,
    "creation_time": datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%d %H:%M:%S"),
    "author": "Nils Niebaum",
    "email": "nils-ole.niebaum@mpimet.mpg.de",
    "institution": "Max Planck Institute for Meteorology",
    "github_repository": "https://github.com/nilsnevertree/sdm-eurec4a",
    "git_commit": get_git_revision_hash(),
}

##### Fit in mass concentration space

In [ ]:
list_parameters = []
fitted_data = []

for cloud_id in tqdm(identified_clusters["cloud_id"]):

    # extract the cloud composite data for the cloud
    cc = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=coarse_composite,
    )
    # cc = cc.expand_dims(cloud_id = [int(cloud_id.values)])

    # use the mass size distribution and the radius
    msd = cc["mass_size_distribution"]  # .sel(radius = slice(15e-6, None))
    radius = cc["radius"].expand_dims(time=cc["time"])  # .sel(radius = slice(15e-6, None))

    # make sure to have the same order of dimensions
    msd = msd.transpose("time", "radius", ...)
    radius = radius.transpose("time", "radius", ...)

    # fit the double log-normal distribution

    double_ln = smodels.CleoDoubleLnNormalFit(
        name="MSD",
        x0=MSDBounds().x0,
        bounds=MSDBounds().bounds,
        t_train=radius.mean("time", skipna=True),
        y_train=msd.mean("time", skipna=True),
        fit_kwargs=dict(
            loss="linear",
            kwargs=dict(
                # parameter_space= 'geometric',
                # x_space = 'ln',
                # density_scaled = False,
            ),
        ),
    )
    double_ln.fit(3)
    double_ln.parameters

    # save the parameters
    list_parameters.append(
        dict_to_DataArray(
            d=double_ln.parameters,
            new_coords=dict(
                cloud_id=[
                    cloud_id,
                ]
            ),
        )
    )
    # predict the number concentration
    dimension, prediction = double_ln.predict(t_test)

    prediction = prediction.expand_dims(cloud_id=(cloud_id,))

    # # we can also use only the radii where we have data:
    # radii_measured = cc["particle_size_distribution"].mean("time") > 0
    # radius_end = cc["radius"].sel(radius=radii_measured).max().values
    # radius_start = cc["radius"].sel(radius=radii_measured).min().values
    # radius_end = 1.5 * radius_end
    # prediction = prediction.where(dimension >= radius_start, other = np.nan)
    # prediction = prediction.where(dimension <= radius_end, other = np.nan)

    fitted_data.append(prediction)


# create a data array with the fitted number concentration
mass_size_distribution_fitted_data = xr.concat(
    fitted_data,
    dim="cloud_id",
)
mass_size_distribution_parameters = xr.concat(
    list_parameters,
    dim="cloud_id",
)

  2%|▏         | 3/154 [00:00<00:06, 24.57it/s]

100%|██████████| 154/154 [00:09<00:00, 16.57it/s]


In [ ]:
details = ""
details += "Parameters of the double log-normal distribution fitted to the mass size distribution.\n"
details += "The fitting function was set in the CleoDoubleLnNormalFit class.\n"
details += "No weighting used in the cost function.\n"
details += "A coarsened input data was used.\n"
details += "The loss function was set to 'linear'.\n"

mass_size_distribution_parameters.attrs = {
    "description": "parameters of a double log-normal distribution fitted to the particle size distribution",
    "details": details,
    "creation_time": datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%d %H:%M:%S"),
    "author": "Nils Niebaum",
    "email": "nils-ole.niebaum@mpimet.mpg.de",
    "institution": "Max Planck Institute for Meteorology",
    "github_repository": "https://github.com/nilsnevertree/sdm-eurec4a",
    "git_commit": get_git_revision_hash(),
}

## Combine the data into one dataset

In [ ]:
# combine the datasets
dataset_fitted = xr.Dataset(
    dict(
        particle_size_distribution=particle_size_distribution_fitted_data,
        weighted3_particle_size_distribution=weighted3_particle_size_distribution_fitted_data,
        weighted2_particle_size_distribution=weighted2_particle_size_distribution_fitted_data,
        mass_size_distribution=mass_size_distribution_fitted_data,
    )
)
dataset_fitted["bin_width"] = w_test
dataset_fitted["bin_width"].attrs = dict(
    long_name="Bin width",
    unit="m",
)

# Fit to the number concentration
dataset_fitted["particle_size_distribution"].attrs = dict(
    long_name="Number concentration", unit="m^{-3} m^{-1}", comment="Fit to the number concentration"
)

dataset_fitted["mass_size_distribution_from_nc"] = (
    msd_from_psd_dataarray(
        da=dataset_fitted["particle_size_distribution"] * dataset_fitted["bin_width"],
    )
    / dataset_fitted["bin_width"]
)
dataset_fitted["mass_size_distribution_from_nc"].attrs = dict(
    long_name="Mass concentration from number concentration",
    unit="kg m^{-3} m^{-1}",
    comment="Fit to the number concentration",
)

# Fit to the number concentration with weight
dataset_fitted["weighted3_particle_size_distribution"].attrs = dict(
    long_name="Weighted number concentration",
    unit="m^{-3} m^{-1}",
    comment="Fit to the number concentration\nWeighted by the cube of the radius",
)

dataset_fitted["mass_size_distribution_from_wnc3"] = (
    msd_from_psd_dataarray(
        da=dataset_fitted["weighted3_particle_size_distribution"] * dataset_fitted["bin_width"],
    )
    / dataset_fitted["bin_width"]
)
dataset_fitted["mass_size_distribution_from_wnc3"].attrs = dict(
    long_name="Mass concentration from weighted number concentration",
    unit="kg m^{-3} m^{-1}",
    comment="Fit to the number concentration\nWeighted by the cube of the radius",
)

# Fit to the number concentration with weight of radius ^ 2
dataset_fitted["weighted2_particle_size_distribution"].attrs = dict(
    long_name="Weighted number concentration",
    unit="m^{-3} m^{-1}",
    comment="Fit to the number concentration\nWeighted by the square of the radius",
)

dataset_fitted["mass_size_distribution_from_wnc2"] = (
    msd_from_psd_dataarray(
        da=dataset_fitted["weighted2_particle_size_distribution"] * dataset_fitted["bin_width"],
    )
    / dataset_fitted["bin_width"]
)
dataset_fitted["mass_size_distribution_from_wnc2"].attrs = dict(
    long_name="Mass concentration from weighted number concentration",
    unit="kg m^{-3} m^{-1}",
    comment="Fit to the number concentration\nWeighted by the square of the radius",
)


# Fit to the mass concentration
dataset_fitted["mass_size_distribution"].attrs = dict(
    long_name="Mass concentration", unit="kg m^{-3} m^{-1}", comment="Fit to the mass concentration"
)

dataset_fitted["particle_size_distribution_from_mc"] = (
    psd_from_msd_dataarray(
        da=dataset_fitted["mass_size_distribution"] * dataset_fitted["bin_width"],
    )
    / dataset_fitted["bin_width"]
)
dataset_fitted["particle_size_distribution_from_mc"].attrs = dict(
    long_name="Number concentration from mass concentration",
    unit="m^{-3} m^{-1}",
    comment="Fit to the mass concentration",
)

dataset_fitted["radius"].attrs.update(coarse_composite["radius"].attrs)

dataset_fitted["radius_micrometer"] = dataset_fitted["radius"] * 1e6
dataset_fitted["radius_micrometer"].attrs = dict(
    long_name="Radius",
    unit="µm",
)

colors = dict(
    particle_size_distribution="orange",
    mass_size_distribution_from_nc="orange",
    weighted3_particle_size_distribution="blue",
    mass_size_distribution_from_wnc3="blue",
    weighted2_particle_size_distribution="red",
    mass_size_distribution_from_wnc2="red",
    particle_size_distribution_from_mc="purple",
    mass_size_distribution="purple",
)

individual parameters

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(9, 6))

for i, ds in enumerate(
    [
        particle_size_distribution_parameters,
        weighted2_particle_size_distribution_parameters,
        weighted3_particle_size_distribution_parameters,
        mass_size_distribution_parameters,
    ]
):
    axs[i, 0].scatter(
        1e6 * ds["mu1"],
        1e6 * ds["mu2"],
        marker=".",
        alpha=0.5,
    )
    axs[i, 0].set_xlabel("mu1")
    axs[i, 0].set_ylabel("mu2")

    axs[i, 1].scatter(
        1e6 * ds["sigma1"],
        1e6 * ds["sigma2"],
        marker=".",
        alpha=0.5,
    )
    axs[i, 1].set_xlabel("sigma1")
    axs[i, 1].set_ylabel("sigma2")

    try:
        axs[i, 2].scatter(
            ds["scale_factor1"],
            ds["scale_factor2"],
            marker=".",
            alpha=0.5,
        )
    except Exception:
        axs[i, 2].scatter(
            ds["scale1"],
            ds["scale2"],
            marker=".",
            alpha=0.5,
        )
    axs[i, 2].set_xlabel("scale1")
    axs[i, 2].set_ylabel("scale2")


fig.tight_layout()

### Distirbution of PSD and MSD

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(13, 5))


# plot the particle size distribution

for key in [
    "particle_size_distribution",
    "particle_size_distribution_from_mc",
    "weighted3_particle_size_distribution",
    "weighted2_particle_size_distribution",
]:
    axs[0].plot(
        dataset_fitted["radius"],
        dataset_fitted[key].T,
        color=colors[key],
        label=key,
        alpha=0.1,
    )

    axs[0].plot(
        coarse_composite["radius"],
        coarse_composite["particle_size_distribution"].mean("time"),
        marker=".",
        linestyle="None",
        color="black",
        zorder=10,
    )

axs[0].set_xscale("log")
# axs[0].set_yscale("log")
axs[0].set_yscale("symlog", linthresh=1e-1, linscale=0.1)
axs[0].set_yticks((1e0, 1e4, 1e8, 1e12, 1e16))
axs[0].set_ylim(0, 1e14)

axs[0].set_xlabel(label_from_attrs(dataset_fitted["radius_micrometer"]))
axs[0].set_ylabel(label_from_attrs(dataset_fitted["particle_size_distribution"]))


# plot the mass size distribution
for key in [
    "mass_size_distribution_from_nc",
    "mass_size_distribution_from_wnc3",
    "mass_size_distribution_from_wnc2",
    "mass_size_distribution",
]:
    axs[1].plot(
        dataset_fitted["radius"],
        dataset_fitted[key].T,
        color=colors[key],
        label=key,
        alpha=0.1,
    )

axs[1].set_xscale("log")
# axs[1].set_yscale("log")
axs[1].set_yscale("symlog", linthresh=1e-10, linscale=0.1)
axs[1].set_yticks((1e-8, 1e-6, 1e-4, 1e-2, 1e0, 1e2, 1e4))
axs[1].set_ylim(0, 1e4)


axs[1].set_xlabel(label_from_attrs(dataset_fitted["radius_micrometer"]))
axs[1].set_ylabel(label_from_attrs(dataset_fitted["mass_size_distribution"]))

fig.suptitle("Comparison of the three different fits")
fig.tight_layout()
fig.savefig(fig_dir / "fit_comparison.png", dpi=300)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(13, 5))


im = axs[0].hist2d(
    np.ravel(dataset_fitted["particle_size_distribution_from_mc"]),
    np.ravel(dataset_fitted["weighted3_particle_size_distribution"]),
    bins=np.geomspace(1e-12, 1e14, 100),
    cmap="plasma",
    norm=mcolors.LogNorm(),
)
axs[0].set_xscale("log")
axs[0].set_yscale("log")
fig.colorbar(im[-1], ax=axs[0])

im = axs[1].hist2d(
    np.ravel(dataset_fitted["mass_size_distribution"]),
    np.ravel(dataset_fitted["mass_size_distribution_from_wnc3"]),
    bins=np.geomspace(1e-14, 6e1, 100),
    cmap="plasma",
    norm=mcolors.LogNorm(vmin=1, vmax=300),
)
axs[1].set_xscale("log")
axs[1].set_yscale("log")
fig.colorbar(im[-1], ax=axs[1])

## Analysis of the fits

### Compare the LWC from the fits to the observations

##### Full radius range

In [ ]:
fig, axs = plt.subplots(ncols=4, figsize=(15, 4.5), sharex=True, sharey=True)

i = 0
for key in (
    "mass_size_distribution_from_nc",
    "mass_size_distribution_from_wnc3",
    "mass_size_distribution_from_wnc2",
    "mass_size_distribution",
):

    lwc = 1e3 * (dataset_fitted[key] * dataset_fitted["bin_width"]).sum("radius")

    axs[i].errorbar(
        x=ds_observations["liquid_water_content"],
        xerr=ds_observations["liquid_water_content_sem"],
        y=lwc,
        yerr=0,
        marker=".",
        linestyle="None",
        label=dataset_fitted[key].attrs["comment"],
        color=colors[key],
        alpha=0.5,
    )
    axs[i].set_title(
        dataset_fitted[key].attrs["comment"],
        color=adjust_lightness_array(
            [
                colors[key],
            ],
            0.75,
        )[0],
    )

    # axs[i].set_title(dataset_fitted[key].attrs['comment'])

    i += 1


for _ax in axs:
    _ax.plot([0, 3], [0, 3], color="black", linestyle="--")
    # _ax.set_aspect('equal')
    _ax.set_xlabel("Observed LWC [g m$^{-3}$]")
    _ax.set_ylabel("Fitted LWC [g m$^{-3}$]")

fig.suptitle("Comparison of fitted and observed LWC for multiple double Log-Normal fits")

fig.tight_layout()

_ax.set_xlim(0, 5)
_ax.set_ylim(0, 5)

fig.savefig(fig_dir / "LWC_fit_comparison.png", dpi=400)

_ax.set_xlim(0, 0.7)
_ax.set_ylim(0, 0.7)

fig.savefig(fig_dir / "LWC_fit_comparison_zoom.png", dpi=400)

##### Radii larger than 50 µm

In [ ]:
fig, axs = plt.subplots(ncols=4, figsize=(15, 4.5), sharex=True, sharey=True)

i = 0
for key in (
    "mass_size_distribution_from_nc",
    "mass_size_distribution_from_wnc3",
    "mass_size_distribution_from_wnc2",
    "mass_size_distribution",
):

    lwc = 1e3 * (dataset_fitted[key] * dataset_fitted["bin_width"]).sel(radius=slice(50e-6, None)).sum(
        "radius"
    )

    axs[i].errorbar(
        x=1e3 * ds_observations["liquid_water_content_50um"],
        xerr=1e3 * ds_observations["liquid_water_content_50um_sem"],
        y=lwc,
        yerr=0,
        marker=".",
        linestyle="None",
        label=dataset_fitted[key].attrs["comment"],
        color=colors[key],
        alpha=0.5,
    )

    axs[i].set_title(
        dataset_fitted[key].attrs["comment"],
        color=adjust_lightness_array(
            [
                colors[key],
            ],
            0.75,
        )[0],
    )
    i += 1

for _ax in axs:
    _ax.plot([0, 3], [0, 3], color="black", linestyle="--")
    # _ax.set_aspect('equal')
    _ax.set_xlabel("Observed LWC [g m$^{-3}$]")
    _ax.set_ylabel("Fitted LWC [g m$^{-3}$]")

fig.suptitle(
    "Comparison of fitted and observed LWC for multiple double Log-Normal fits\n Only Radii above 50 µm"
)

fig.tight_layout()

_ax.set_xlim(0, 5)
_ax.set_ylim(0, 5)

fig.savefig(fig_dir / "LWC_fit_comparison-50um.png", dpi=400)

_ax.set_xlim(0, 0.7)
_ax.set_ylim(0, 0.7)

fig.savefig(fig_dir / "LWC_fit_comparison-50um-zoom.png", dpi=400)

### Compare NBC from the fits with the observations

##### Full radius range

In [ ]:
fig, axs = plt.subplots(ncols=4, figsize=(15, 4.5), sharex=True, sharey=True)

i = 0
for key in (
    "particle_size_distribution",
    "weighted3_particle_size_distribution",
    "weighted2_particle_size_distribution",
    "particle_size_distribution_from_mc",
):

    nbc = (dataset_fitted[key] * dataset_fitted["bin_width"]).sum("radius")

    axs[i].errorbar(
        x=ds_observations["particle_size_distribution"],
        xerr=ds_observations["particle_size_distribution_sem"],
        y=nbc,
        yerr=0,
        marker=".",
        linestyle="None",
        label=dataset_fitted[key].attrs["comment"],
        color=colors[key],
        alpha=0.5,
    )
    axs[i].set_title(
        dataset_fitted[key].attrs["comment"],
        color=adjust_lightness_array(
            [
                colors[key],
            ],
            0.75,
        )[0],
    )

    # axs[i].set_title(dataset_fitted[key].attrs['comment'])

    i += 1


for _ax in axs:

    _ax.plot([0, 8e8], [0, 8e8], color="black", linestyle="--")
    # _ax.set_aspect('equal')
    _ax.set_xlabel("Observed NBC [m$^{-3}$]")
    _ax.set_ylabel("Fitted NBC [m$^{-3}$]")

fig.suptitle("Comparison of fitted and observed NBC for multiple double Log-Normal fits")

fig.tight_layout()

_ax.set_xlim(0, 8e8)
_ax.set_ylim(0, 8e8)

fig.savefig(fig_dir / "NBC_fit_comparison.png", dpi=400)

_ax.set_xlim(0, 2.5e8)
_ax.set_ylim(0, 2.5e8)

fig.savefig(fig_dir / "NBC_fit_comparison_zoom.png", dpi=400)

##### Radii larger than 50 µm

In [ ]:
fig, axs = plt.subplots(ncols=4, figsize=(15, 4.5), sharex=True, sharey=True)

i = 0
for key in (
    "particle_size_distribution",
    "weighted3_particle_size_distribution",
    "weighted2_particle_size_distribution",
    "particle_size_distribution_from_mc",
):

    nbc = (dataset_fitted[key].sel(radius=slice(50e-6, None)) * dataset_fitted["bin_width"]).sum(
        "radius"
    )

    axs[i].errorbar(
        x=ds_observations["particle_size_distribution_50um"],
        xerr=ds_observations["particle_size_distribution_50um_sem"],
        y=nbc,
        yerr=0,
        marker=".",
        linestyle="None",
        label=dataset_fitted[key].attrs["comment"],
        color=colors[key],
        alpha=0.5,
    )
    axs[i].set_title(
        dataset_fitted[key].attrs["comment"],
        color=adjust_lightness_array(
            [
                colors[key],
            ],
            0.75,
        )[0],
    )

    # axs[i].set_title(dataset_fitted[key].attrs['comment'])

    i += 1


for _ax in axs:

    _ax.plot([0, 2e5], [0, 2e5], color="black", linestyle="--")
    # _ax.set_aspect('equal')
    _ax.set_xlabel("Observed NBC [m$^{-3}$]")
    _ax.set_ylabel("Fitted NBC [m$^{-3}$]")

fig.suptitle(
    "Comparison of fitted and observed NBC for multiple double Log-Normal fits\n Only Radii above 50 µm"
)

fig.tight_layout()

_ax.set_xlim(0, 20e3)
_ax.set_ylim(0, 20e3)


# _ax.set_xscale("log")
# _ax.set_yscale("log")
fig.savefig(fig_dir / "NBC_fit_comparison-50um.png", dpi=400)

_ax.set_xlim(0, 5e3)
_ax.set_ylim(0, 5e3)

fig.savefig(fig_dir / "NBC_fit_comparison-50um-zoom.png", dpi=400)

### Plot distributions of some random clouds

In [ ]:
small_cloud_ids = np.random.choice(identified_clusters["cloud_id"], 3, replace=False)

large_cloud_ids = identified_clusters["cloud_id"].where(
    identified_clusters["liquid_water_content"] / identified_clusters["duration"].dt.seconds > 0.5,
    drop=True,
)
large_cloud_ids = np.random.choice(large_cloud_ids, 3, replace=False)

cloud_ids = np.concatenate([small_cloud_ids, large_cloud_ids])

ncols_nrows = ncols_nrows_from_N(len(cloud_ids))

##### Particle size distirbution

In [ ]:
fig, axs = plt.subplots(figsize=(10, 7), sharex=True, sharey=True, **ncols_nrows)

# plot the PSDs of the selected clouds
for i, cloud_id in enumerate(cloud_ids):
    _ax = axs.flatten()[i]
    cc = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=coarse_composite,
    )

    lwc_mean, lwc_sem = mean_and_stderror_of_mean(cc["liquid_water_content"], dims=("time",))

    _ax.set_title(
        f"Cloud ID: {cloud_id:.0f}\n{lwc_mean.values:.2f} ± {lwc_sem.values:.2f} g" + "$m^{-3}$"
    )

    observations = cc["particle_size_distribution"]

    _ax.plot(
        1e6 * observations["radius"],
        observations.mean("time"),
        marker=".",
        linestyle="None",
        color="black",
    )

    for key in (
        "particle_size_distribution",
        "weighted3_particle_size_distribution",
        "weighted2_particle_size_distribution",
        "particle_size_distribution_from_mc",
    ):

        fit = dataset_fitted[key].sel(cloud_id=cloud_id)
        _ax.plot(
            dataset_fitted["radius_micrometer"],
            fit,
            color=colors[key],
            linestyle="-",
            label=fit.attrs["comment"],
        )

    _ax.set_xscale("log")
    _ax.set_yscale("symlog", linthresh=1e4, linscale=0.1)
    _ax.set_yticks([0, 1e6, 1e9, 1e12])
# axs.flatten()[0].legend()
_ax.set_ylim(0, None)

fig.supxlabel(label_from_attrs(dataset_fitted["radius_micrometer"]))
fig.supylabel(label_from_attrs(dataset_fitted["particle_size_distribution"]))

fig.suptitle("Comparison of fitted and observed PSDs for multiple double Log-Normal fits")
fig.tight_layout()

fig.savefig(fig_dir / "PSD_fit_comparison.png", dpi=400)

##### Mass size distribution

In [ ]:
fig, axs = plt.subplots(figsize=(10, 7), sharex=True, sharey=True, **ncols_nrows)

# plot the PSDs of the selected clouds
for i, cloud_id in enumerate(cloud_ids):
    _ax = axs.flatten()[i]
    cc = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=coarse_composite,
    )

    lwc_mean, lwc_sem = mean_and_stderror_of_mean(cc["liquid_water_content"], dims=("time",))

    _ax.set_title(
        f"Cloud ID: {cloud_id:.0f}\n{lwc_mean.values:.2f} ± {lwc_sem.values:.2f} g" + "$m^{-3}$"
    )

    observations = cc["mass_size_distribution"]

    _ax.plot(1e6 * cc["radius"], observations.mean("time"), marker=".", linestyle="None", color="black")

    for key in (
        "mass_size_distribution",
        "mass_size_distribution_from_wnc3",
        "mass_size_distribution_from_wnc2",
        "mass_size_distribution_from_nc",
    ):

        fit = dataset_fitted[key].sel(cloud_id=cloud_id)
        _ax.plot(
            dataset_fitted["radius_micrometer"],
            fit,
            color=colors[key],
            linestyle="-",
            alpha=0.8,
        )

    _ax.set_xscale("log")
    _ax.set_yscale("symlog", linthresh=1e-6, linscale=0.1)
    _ax.set_yticks([0, 1e-3, 1e0, 1e3])
    _ax.set_xlim(1, 3e3)
_ax.set_ylim(0, None)

fig.supxlabel("Radius [$µm$]")
fig.supylabel(label_from_attrs(dataset_fitted["mass_size_distribution"]))

fig.suptitle("Comparison of fitted and observed MSDs for multiple double Log-Normal fits")
fig.tight_layout()

fig.savefig(fig_dir / "MSD_fit_comparison.png", dpi=400)

##### Linear axes - mass size distribution

In [ ]:
fig, axs = plt.subplots(figsize=(10, 7), sharex=True, sharey=False, **ncols_nrows)

# plot the PSDs of the selected clouds
for i, cloud_id in enumerate(cloud_ids):
    _ax = axs.flatten()[i]
    cc = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=coarse_composite,
    )

    lwc_mean, lwc_sem = mean_and_stderror_of_mean(cc["liquid_water_content"], dims=("time",))

    _ax.set_title(
        f"Cloud ID: {cloud_id:.0f}\n{lwc_mean.values:.2f} ± {lwc_sem.values:.2f} g" + "$m^{-3}$"
    )

    observations = cc["mass_size_distribution"] * cc["bin_width"]

    _ax.plot(
        1e3 * observations["radius"],
        1e3 * observations.mean("time"),
        marker=".",
        linestyle="None",
        color="black",
    )

    for key in (
        "mass_size_distribution_from_nc",
        "mass_size_distribution_from_wnc3",
        "mass_size_distribution_from_wnc2",
        "mass_size_distribution",
    ):

        fit = dataset_fitted[key].sel(cloud_id=cloud_id)
        _ax.plot(
            1e3 * dataset_fitted["radius"],
            1e3 * fit * dataset_fitted["bin_width"],
            color=colors[key],
            linestyle="-",
            label=fit.attrs["comment"],
            alpha=0.8,
        )
    _ax.set_xlim([0, 3])

    # _ax.set_xscale('log')
# _ax.set_yscale('symlog', linthresh = 1e4)
for _ax in axs[0]:
    _ax.set_ylim(0, 0.008)

for _ax in axs[1]:
    _ax.set_ylim(0, 0.15)


fig.supxlabel("Radius [$mm$]")
fig.supylabel("Mass concentration [$g m^{-3}$]")

fig.suptitle("Comparison of fitted and observed MSDs for multiple double Log-Normal fits")
fig.tight_layout()

fig.savefig(fig_dir / "MSD_fit_comparison_linear.png", dpi=400)

# Decide for the correct fits to use

We think, that the best solution would be the:
- <span style="color:#8A2BE2">**Mass Concentration fit**</span>

The easiest and still good estimate is the 
- <span style="color:#6495ED">**Number Concentration fit with weight to $r^3$**</span>

In [ ]:
psd_parameters = weighted3_particle_size_distribution_parameters
attrs = psd_parameters.attrs
params1_linear_space = (
    smodels.GeometricMuSigmaScaleLog(
        geometric_mu_l=psd_parameters["mu1"],
        geometric_std_dev=psd_parameters["sigma1"],
        scale_l=psd_parameters["scale_factor1"],
    )
    .standardize()
    .get_geometric_parameters_linear(
        dict_keys=("geometric_mean1", "geometric_std_dev1", "scale_factor1")
    )
)

params2_linear_space = (
    smodels.GeometricMuSigmaScaleLog(
        geometric_mu_l=psd_parameters["mu2"],
        geometric_std_dev=psd_parameters["sigma2"],
        scale_l=psd_parameters["scale_factor2"],
    )
    .standardize()
    .get_geometric_parameters_linear(
        dict_keys=("geometric_mean2", "geometric_std_dev2", "scale_factor2")
    )
)

psd_parameters_linear_space = xr.Dataset(dict(**params1_linear_space, **params2_linear_space))

mapping = dict(
    geometric_mean1="geometric_mean1",
    geometric_mean2="geometric_mean2",
    geometric_std_dev1="geometric_std_dev1",
    geometric_std_dev2="geometric_std_dev2",
    scale_factor1="scale_factor1",
    scale_factor2="scale_factor2",
)

psd_parameters_linear_space.attrs.update(attrs)
psd_parameters_linear_space.attrs["parameter_space"] = "geometric"
psd_parameters_linear_space.attrs["independent_space"] = "linear"

psd_parameters.attrs["parameter_space"] = "geometric"
psd_parameters.attrs["independent_space"] = "log"

## Store output data

In [ ]:
data_dir = RepositoryPath("levante").data_dir
output_dir = data_dir / Path("model/input_v4.2")
output_dir.mkdir(exist_ok=True)

dataset_fitted.to_netcdf(output_dir / Path("fitted_distributions.nc"))

mass_size_distribution_parameters.to_netcdf(output_dir / Path("mass_size_distribution_parameters.nc"))

psd_parameters.to_netcdf(output_dir / Path("particle_size_distribution_parameters.nc"))

psd_parameters_linear_space.to_netcdf(
    output_dir / Path("particle_size_distribution_parameters_linear_space.nc")
)

ds_observations.to_netcdf(output_dir / Path("lwc_and_nbc_cloud_composite.nc"))

In [ ]:
p = double_ln.parameters

np.testing.assert_array_almost_equal(
    smodels.double_ln_normal_distribution(t_test, *p.values()),
    smodels.double_log_normal_distribution_all(
        t_test, *p.values(), parameter_space="geometric", x_space="ln"
    ),
)

In [ ]:
p = weighted3_particle_size_distribution_parameters

p = dict(
    mu1=p["mu1"],
    mu2=p["mu2"],
    sigma1=p["sigma1"],
    sigma2=p["sigma2"],
    scale_factor1=p["scale_factor1"],
    scale_factor2=p["scale_factor2"],
)

p_all = dict(
    mu1=p["mu1"],
    mu2=p["mu2"],
    sigma1=p["sigma1"],
    sigma2=p["sigma2"],
    scale1=p["scale_factor1"],
    scale2=p["scale_factor2"],
)

a = smodels.double_log_normal_distribution_all(
    t_test, **p_all, parameter_space="geometric", x_space="ln"
)

l = smodels.double_ln_normal_distribution(t_test, **p).T

In [ ]:
l